In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widjets
%matplotlib inline

import numpy as np
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt

import os
import shutil
import cv2
from copy import deepcopy

from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from time import time
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn import metrics
import json
from pathlib import Path
from scipy.optimize import linear_sum_assignment

In [3]:
video = cv2.VideoCapture(r'vids/campus4-c0.avi')
video.set(cv2.CAP_PROP_BUFFERSIZE, 5)
fps = int(video.get(cv2.CAP_PROP_FPS)/2)
max_frame = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(f'fps: {fps}, max_frame: {max_frame}')

fps: 12, max_frame: 2000


In [4]:
# ЧИТААААТЬ

frames_dict = {}

for curr_frame in tqdm_notebook(range(max_frame)):
#     time_st = time()
#     video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
#     time_set = time() - time_st
    
    time_st = time()
    success, frame = video.read()
    time_read = time() - time_st
    
    time_st = time()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames_dict[curr_frame] = frame
#         frames_dict.append({'frame_num':curr_frame, 'frame':frame})
    time_save = time() - time_st
    
    print(f"read: {time_read}, save: {time_save}", end='\r')


c:\python37\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


read: 0.0, save: 0.09042969, save: 0.0009918212890625047


In [5]:
p = r'vids/campus4-c0.csv'
DD = pd.read_csv(str(p), converters={'bboxes': eval})

In [6]:
def get_img_w_rect(test_img, bboxes):
    out_img = test_img.copy()
    color = (0, 255, 0)
    thickness = 2
    for i, bbox in enumerate(bboxes):
        x0, y0, x1, y1 = bbox
        out_img = cv2.rectangle(out_img, (x0,y0), (x1,y1), color, thickness)

    return out_img

In [7]:
@interact(i=(0, len(DD)-1, 1))
def test(i):
    key = DD.iloc[i]['frame_num']
    bboxes = DD.iloc[i]['bboxes']
    image = frames_dict[key]
    img = get_img_w_rect(image, bboxes)
    plt.imshow(img)

interactive(children=(IntSlider(value=859, description='i', max=1719), Output()), _dom_classes=('widget-intera…

In [8]:
DD.iloc[143]

frame_num                      143
bboxes       [[133, 85, 179, 206]]
Name: 143, dtype: object

# SORT

In [9]:
# # clone sort algorytm
# !git clone https://github.com/abewley/sort.git sort
# !pip install -r requirements.txt

In [10]:
from sort import *

In [11]:
video_names = ['campus4-c0', 'campus4-c1', 'campus4-c2']

mot_tracker = Sort()      # Tracker using SORT Algorithm
save_path = r'Z:\test tasks\EORA\result'

In [12]:
h,w = frames_dict[123].shape[:2]
# output_video = cv2.VideoWriter('output.mp4', -1, 24.0, (w,h))
output_video = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 24,(w,h))

In [13]:
new_frames = []
for idx, row in tqdm(DD.iterrows()):
    frame_num = row['frame_num']
    bboxes = row['bboxes']
    det_img = frames_dict[frame_num].copy()
    
    det_list = np.array([bbox+[1.] for bbox in bboxes])
    if len(det_list)==0:
        det_list = np.empty((0, 5))

    track_bbs_ids = mot_tracker.update(det_list)

    mot_imgid = str(frame_num)
    newname = os.path.join(save_path, mot_imgid + '.png')
    
    # draw rectangles
    for row in track_bbs_ids:  
        x, y, x2, y2, _id = list(map(int, row))
        cv2.rectangle(det_img, (x, y), (x2, y2), (0, 255, 0), 4)
        _id = str(_id)
        cv2.putText(det_img, '#'+_id, (x+5, y-10), 0,0.6,(0,255,0),thickness=2)
    
    
    #### show not tracked guys
    IoU = -iou_batch(det_list, track_bbs_ids)
    prev_indices, boxes_indices = linear_sum_assignment(IoU)
#     print(prev_indices, boxes_indices)

    # leave non-corresponding bboxes only 
    mask = np.ones(det_list.shape[0])
    mask[prev_indices] = 0
    not_tracked_bboxes = det_list[mask.astype(bool)]
    
    for row in not_tracked_bboxes:  
        x, y, x2, y2, _id = list(map(int, row))
        cv2.rectangle(det_img, (x, y), (x2, y2), (0, 0, 255), 4)
        
    
    output_video.write(det_img)
    cv2.imwrite(newname,det_img)
output_video.release()

1720it [00:11, 155.67it/s]
